In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import csv
import os
import numpy as np
import collections

import scipy.io.wavfile
from scipy.io import loadmat
import collections

import sys, os
sys.path.append(os.path.expanduser('~/projects/engaged_hackathon/'))
from engaged.features import features as engaged_features
from engaged.features import frequency

In [2]:
# we've already created the spectrograms. What we want to do now is to do the full multilabel classification
# let's just randomly extract slices...?
# then each slice has one or more labels

In [3]:
# let's now for each 
import helpers
all_annotations, fannotations = helpers.load_annotations()

In [4]:
# find out all the classes
print all_annotations[0]

# maintain a common list of classes. This may come back to bite us in the future, but lets see...
classes = sorted(list(set([ann['Label'] for ann in all_annotations])))
print classes

{'MeanAmp': 0.0, 'LabelStartTime_Seconds': 0.0, 'MaximumFreq_Hz': 0.0, 'MaxAmp': 0.0, 'Spec_NStep': 0.01, 'MinAmp': 0.0, 'LabelArea_Datapoints_Threshold': 0.0, 'LabelArea_DataPoints': 0.0, 'Label': 'mix traffic', 'Spec_x2': 0.0, 'length': 59.99, 'AmpSD': 0.0, 'Spec_y1': 0.0, 'LabelEndTime_Seconds': 59.99, 'Spec_NWin': 0.03, 'Spec_x1': 0.0, 'Filename': 'E47EN-013527_20131008_0708.wav', 'LabelTimeStamp': '2015-03-11T13:59:59.413000', 'Spec_y2': 0.0, 'MinimumFreq_Hz': 0.0}
['airplane', 'anthropogenic unknown', 'applause', 'bat', 'beep', 'bells', 'bird', 'braking', 'bus emitting', 'coughing', 'dog bark', 'electrical', 'footsteps', 'fox', 'grey squirrel', 'horn', 'invertebrate', 'metal', 'mix traffic', 'mower', 'music', 'rain', 'siren', 'startthecar', 'unknown sound', 'voices', 'whistle', 'wing beats']


In [5]:
# set a sample rate - the labels can be resampled later
sr = 44000
file_labels = {}
spec_path = '/home/michael/projects/engaged_hackathon_data/detection/spectrograms/'
slices_per_file = 1000
hww = 9
spec_size = 128

# # I will pre-allocate a numpy array for memory efficiency
# numpy_shape = (slices_per_file * len(fannotations), 1, spec_size, 2*hww+1)
# if np.prod(np.array(numpy_shape)) > 4e9: raise Exception("TOO large!")
# if 'all_slices' in vars(): del all_slices
# all_slices = np.zeros(numpy_shape) * np.nan
savedir = '/home/michael/projects/engaged_hackathon_data/multilabel_classification/'

In [6]:
# # generate labels for each file...
# for count, (fname, v) in enumerate(fannotations.iteritems()):
    
#     labels = collections.defaultdict(lambda : np.zeros(sr*60))
    
#     for ann in v:
#         start = int(ann['LabelStartTime_Seconds'] * float(sr))
#         end = int(ann['LabelEndTime_Seconds'] * float(sr))
#         labels[ann['Label']][start:end] = 1

#     # load in the spectrogram
#     D = scipy.io.loadmat(spec_path + fname[:-4] + '_spec_' + str(spec_size) + '.mat')
#     spec = D['spectrogram']
#     spec_sr = D['sample_rate']
    
#     # choose random locations in the spectrogram
#     locations = np.random.choice(spec.shape[1], slices_per_file, replace=False)
        
#     # lets extract suitable slices from this dataset
#     slices = helpers.extract_1d_patches(spec, locations, hww)
#     slices = slices.reshape((slices.shape[0], 1, slices.shape[1], slices.shape[2]))
        
#     # now give these slices a label...
#     keys = labels.keys()
#     rescaled_locations = (sr * (locations / float(spec_sr))).astype(int)
#     location_labels = [labels[key][rescaled_locations] for key in classes]
#     location_labels = np.vstack(location_labels)
    
#     D = dict(
#         slices=slices, 
#         location_labels=location_labels,
#         class_names=classes,
#         locations_spec=locations,
#         spec_sr=spec_sr,
#         locations_wav=rescaled_locations,
#         wav_sr=sr)
#     savepath = savedir + 'slices/' + fname.replace('.wav', '.mat')
    
#     scipy.io.savemat(savepath, D, do_compression=True)
    
#     if count % 10 == 0:
#         print count,

In [7]:
# now should do the train/test split and then combine labels and slices...
from sklearn.cross_validation import train_test_split

split_files = {}
split_files['train'], split_files['test'] = train_test_split(
    fannotations.keys(), train_size=0.7, test_size=0.3, random_state=10)

scipy.io.savemat(savedir + 'split_files.mat', split_files)

In [11]:
# now load and combine the training and test data separately

# creating empty array
def create_slice_array(num_slices, spec_height, hww):
    numpy_shape = (num_slices, 1, spec_height, 2*hww+1)
    if np.prod(np.array(numpy_shape)) > 4e9: raise Exception("TOO large!")
    
    all_slices = np.zeros(numpy_shape, np.float32) * np.nan
    return all_slices


for split_name in ['train', 'test']:

    print "\n" + split_name
    
    split = split_files[split_name]

    # clear the variable first to prevent memory overflow
    if 'all_slices' in vars(): del all_slices
    if 'all_labels' in vars(): del all_labels
    all_slices = create_slice_array(
        slices_per_file * len(split), spec_size, hww)
    all_labels = create_slice_array(
        slices_per_file * len(split), len(classes), 0)
    
    print all_slices.shape, all_labels.shape, all_slices.dtype
    
    # now load in each training slice file one at a time, and add to the slice array...
    idx = 0
    temp_labs = []
    for count, fname in enumerate(split):
        D = scipy.io.loadmat(
            savedir + 'slices/' + fname.replace('.wav', '.mat'))

        # add to the slice array
        all_slices[idx:idx+D['slices'].shape[0]] = D['slices']

        # add to the label array
        all_labels[idx:idx+D['slices'].shape[0], ...] = \
            D['location_labels'].T.reshape((-1, 1, len(classes), 1)).astype(np.float32)

        idx += D['slices'].shape[0]
        
        del D

        if count % 10 == 0:
            print count,

    # write to disk...
    D = dict(
        slices=all_slices.astype(np.float32),
        labels=all_labels.astype(np.int32),
        class_names=classes)
    print "WARNING - not saving"
#     helpers.savemat_large(savedir + 'all_' + split_name + '.mat', D, do_compression=False)

 
train
(253000, 1, 128, 19) (253000, 1, 28, 1) float32
(1000, 1, 128, 19)
0 (1000, 1, 128, 19)
(1000, 1, 128, 19)
(1000, 1, 128, 19)
(1000, 1, 128, 19)
(1000, 1, 128, 19)
(1000, 1, 128, 19)
(1000, 1, 128, 19)
(1000, 1, 128, 19)
(1000, 1, 128, 19)
(1000, 1, 128, 19)
10 (1000, 1, 128, 19)
(1000, 1, 128, 19)
(1000, 1, 128, 19)
(1000, 1, 128, 19)
(1000, 1, 128, 19)
(1000, 1, 128, 19)
(1000, 1, 128, 19)
(1000, 1, 128, 19)
(1000, 1, 128, 19)
(1000, 1, 128, 19)
20 (1000, 1, 128, 19)
(1000, 1, 128, 19)
(1000, 1, 128, 19)
(1000, 1, 128, 19)
(1000, 1, 128, 19)
(1000, 1, 128, 19)
(1000, 1, 128, 19)
(1000, 1, 128, 19)
(1000, 1, 128, 19)
(1000, 1, 128, 19)
30

KeyboardInterrupt: 